In [1]:
import os
from os import makedirs, path as op
import json
from typing import Collection, Tuple 
from pystac_client import Client
import planetary_computer as pc
from rio_tiler.io import COGReader
from shapely.geometry import shape

In [2]:
# !pip install rio_tiler -U

In [3]:
def download_NAIP(item, fn, area_of_interest):
    """
    Download NAIP imagery from Planetary Computer
    
    Parameters:
    ___

    inputs:
        item: specific item in the STAC collection,
        fn: given file name
        area_of_interest: geometry of the AOI
    
    Returns:
       (None): writen COG of NAIP imagery that intersect with the given AOI
    """
    print(item.datetime)
    href = pc.sign(item.assets["image"].href)
    with COGReader(href) as cog:
        data = cog.feature(area_of_interest, max_size=None, indexes=(1, 2, 3, 4))
    
    with open(fn, "wb") as f:
        img = data.render(img_format="GTiff", add_mask=False)
        f.write(img)

In [4]:
def main(aoi, date_range, out_dir):
    
    """
    Download NAIP imagery from Planetary Computer
    
    Parameters:
    ___

    inputs:
        aoi: the path to the aoi,
        date_range: given date range to download images, e.g. 2010-01-01/2021-12-01
        out_dir: given output direct to save imagery
    
    Returns:
       (None): all writen COG of NAIP imagery that intersect with the given AOIs
    """
        
    catelog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    #read in aoi
    with open(aoi) as f:
        feature = json.load(f)["features"]
        # assuming this is only one geomery feature of an bounding box
        area_of_interest = feature[0]["geometry"]
    search_imagery = catelog.search(
        collections=["naip"], intersects=area_of_interest, datetime=date_range
    )
    items = list(search_imagery.get_items())
    print(f"{len(items)} items found in the {date_range} range for {aoi}!")
    for item in items:
        if not op.exists(out_dir):
            makedirs(out_dir)
        fn = f"{out_dir}/{str(item.datetime)[:10]}_naip_{aoi}.tif"
        download_NAIP(item, fn, area_of_interest)
 

In [5]:
aois = ["aoi0_bounds.geojson", "aoi1_bounds.geojson", "aoi2_bounds.geojson"]

In [21]:
date_range="2010-01-01/2021-12-01"

In [22]:
out_dir="naip_downloaded_20211020"
for aoi in aois:
    main(aoi, date_range, out_dir)

4 items found in the 2010-01-01/2021-12-01 range for aoi0_bounds.geojson!
2018-07-06 00:00:00+00:00
2016-08-03 00:00:00+00:00
2014-06-28 00:00:00+00:00
2012-06-29 00:00:00+00:00
8 items found in the 2010-01-01/2021-12-01 range for aoi1_bounds.geojson!
2018-07-07 00:00:00+00:00
2018-07-07 00:00:00+00:00
2016-08-03 00:00:00+00:00
2016-08-03 00:00:00+00:00
2014-06-28 00:00:00+00:00
2014-06-28 00:00:00+00:00
2012-07-02 00:00:00+00:00
2012-06-29 00:00:00+00:00
4 items found in the 2010-01-01/2021-12-01 range for aoi2_bounds.geojson!
2018-07-07 00:00:00+00:00
2016-08-03 00:00:00+00:00
2014-06-28 00:00:00+00:00
2012-06-29 00:00:00+00:00
